# PYTHON-17. Как получать данные из веб-источников и API

In [1]:
import pandas as pd
import numpy as np

## 2. Веб-запросы

Процесс получения/извлечения информации с веб-ресурсов в интернете называется web-scraping (рус. веб-скрейпинг/веб-скрапинг). Веб-скрапинг может быть проделан вручную пользователем компьютера, однако этот термин обычно связывают с автоматизированными процессами, реализованными с помощью кода.

Какие данные можно извлечь в процессе веб-скрапинга?

* цены на товары конкурентов для оптимизации своей стратегии ценообразования;
* сообщения в социальных медиа, по которым можно отслеживать тренды в той или иной области;
* отзывы о товарах/услугах компании на различных площадках, которые можно впоследствии анализировать;
* контактные данные пользователей соцсетей или форумов для дальнейшего взаимодействия с этими пользователями;
и т.д.

Для того чтобы веб-скрапинг был возможен, необходимо подключение компьютера, который будет задействован в этом процессе, к сети Интернет.

Интернет — это глобальная информационная сеть, которая позволяет компьютерам по всему миру обмениваться информацией. Один компьютер (называемый клиентом) отправляет запрос в определённом формате другому компьютеру (называемому сервером) и получает ответ (текст, изображение, видео и.т.д.).

Клиент и сервер взаимодействуют между собой, обмениваясь одиночными сообщениями (не потоком данных) посредством сетевых протоколов, которые формализуют общение между ними. В настоящее время повсеместно используемый протокол в интернете, позволяющий клиенту получать различные ресурсы (например, HTML-документы), — это протокол __HTTP__.

Запрос, отправляемый клиентом с использованием протокола HTTP, состоит из нескольких элементов:

* адрес, по которому идёт обращение (например, www.google.com);
* техническая информация, например метод запроса;
* дополнительные данные, например если загружается (передаётся) изображение.
Адрес — это URL, Uniform Resource Locator (с англ. Унифицированный Указатель Ресурса).


Ответ, в свою очередь, состоит из следующих элементов:

* код статуса ответа: например, 200 («успешно»), 404 («не найден») и т. д. (более полный список кодов статуса ответа можете посмотреть, перейдя по [ссылке](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP));
* текст в запрошенном формате (HTML, XML, JSON и т. д.) или мультимедийные файлы;
* прочая техническая информация.

### Методы запросов в протоколе HTTP

Для того чтобы указать серверу на то, какое действие мы хотим произвести с ресурсом, в протоколе HTTP используются так называемые методы. В HTTP существует несколько методов, которые описывают действия с ресурсами. Чаще всего используются GET и POST.

__GET — ПОЛУЧЕНИЕ РЕСУРСА__
Метод GET запрашивает информацию из указанного источника и не может влиять на его содержимое. Запрос доступен для кэширования данных (то есть для сохранения, восстановления и дальнейшего использования) и добавления в закладки. Длина запроса ограничена (максимальная длина — 2048 символов).

Пример GET-запроса, отправляемого через адресную строку браузера:http://site.ru/page.php?name=dima&age=27

__POST — СОЗДАНИЕ РЕСУРСА__
Метод POST используется для отправки данных, которые могут оказывать влияние на содержимое ресурса. В отличие от метода GET, запросы POST не могут быть кэшированы, они не остаются в истории браузера и их нельзя добавить в закладки. Длина запроса POST не ограничивается.

Пример POST-запроса, отправляемого через форму запроса:

POST / HTTP/1.0\r\n
Host: www.site.ru\r\n
Referer: http://www.site.ru/index.html\r\n
Cookie: income=1\r\n
Content-Type: application/x-www-form-urlencoded\r\n
Content-Length: 35\r\n
\r\n
login=Dima&password=12345

Подробнее про методы HTTP можете прочитать, перейдя по этой [ссылке](https://developer.mozilla.org/ru/docs/Web/HTTP/Methods).



## 3. Библиотека requests

В стандартной библиотеке Python для отправки веб-запросов существует функция urllib2, но большинство разработчиков используют стороннюю библиотеку requests (c англ. запросы), потому что её работа более стабильна, а созданный с её помощью код получается проще. Поэтому мы будем работать с библиотекой requests, а urllib2 рассматривать не будем.

*Разработаем код, так называемый скрипт (англ. script, рус. сценарий), — небольшую программу, которая содержит последовательность действий для автоматического выполнения задачи.*

С помощью скрипта мы будем в удобном виде выгружать информацию по курсам валют с заранее выбранного сайта.

Один из сайтов в интернете, на котором информация о курсах валют дублирует информацию с сайта [Центрального Банка России](https://www.cbr.ru/), — ресурс [Курсы валют ЦБ РФ в XML и JSON](https://www.cbr-xml-daily.ru/). На данном ресурсе информация о курсах валют представлена в разных форматах, в том числе и в структурированном JSON-формате, методы работы с которым мы изучили в одном из предыдущих модулей.

Перед началом работы библиотеку requests потребуется установить.

In [2]:
!pip install requests

Как только библиотека установлена, импортируем её и отправим наш первый запрос к ресурсу Курсы валют ЦБ РФ в XML и JSON. Используем метод get() из библиотеки requests, передав ему соответствующий URL —  https://www.cbr-xml-daily.ru/daily_json.js

In [3]:
import requests
url = 'https://www.cbr-xml-daily.ru/daily_json.js' # значение url определяем в виде строки
response = requests.get(url) # Делаем GET-запрос к ресурсу и результат ответа сохраняем в переменной response

In [4]:
# Проверим ответ сервера — содержимое переменной response:
print(response)

<Response [200]>


Мы получили объект ответа Response, который содержит всю нужную нам информацию. По умолчанию в квадратных скобках на экран выводится код статуса ответа. В данном случае он равен 200 — то есть запрос был корректным и сервер отдал нам нужную информацию. Значение кода статуса 404 означало бы, что страница по указанному адресу не найдена, а значение 403 — что синтаксис GET-запроса неверный.

In [5]:
# Код ответа в виде числовой переменной можно получить с помощью метода status_code:
print(response.status_code)

200


Задание 2.1:

Вы уже импортировали модуль requests в ваш код. Напишите строку кода, при выполнении которой будет сделан GET-запрос к ресурсу https://www.cbr-xml-daily.ru/daily.xml, а результат ответа будет сохранён в переменной response:

In [7]:
response = requests.get('https://www.cbr-xml-daily.ru/daily.xml')
print(response.status_code)

200


### РАБОТАЕМ С ОТВЕТОМ

Мы сделали запрос и получили корректный ответ (код статуса — 200). Дальнейшую работу производим с результатом запроса к ресурсу Курсы валют ЦБ РФ в XML и JSON.

Как получить доступ ко всей информации, которую содержит ответ?

Текст ответа хранится в атрибуте text. Выведем значение атрибута на экран и посмотрим на его содержимое:

In [9]:
url = 'https://www.cbr-xml-daily.ru/daily_json.js' 
response = requests.get(url)

print(response.text)

{
    "Date": "2023-07-04T11:30:00+03:00",
    "PreviousDate": "2023-07-01T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2023\/07\/01\/daily_json.js",
    "Timestamp": "2023-07-03T20:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 59.5265,
            "Previous": 58.5989
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 52.5444,
            "Previous": 51.9908
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 113.5684,
            "Previous": 111.5234
 

Как правило, при работе над реальным проектом на этапе получения данных мы уже понимаем, с какими форматами данных нам придётся работать. На предлагаемом для работы ресурсе информация есть как в JSON-формате, так и в XML. По нашему запросу ресурс возвращает информацию в JSON-формате, однако в настоящий момент результат хранится как единая строка. Проверить тип данных полученного ответа можно, воспользовавшись функцией type().

In [12]:
print(f'Тип данных полученного ответа - {type(response)}')

Тип данных полученного ответа - <class 'requests.models.Response'>


Для того чтобы удобно было работать с полученной информацией, нам необходимо преобразовать строку в словарь. В объект ответа Response  из библиотеки requests уже встроен метод json(). 

Импортируем функцию pprint(), применим к полученному ответу метод json() и выведем полученный результат на экран:

In [13]:
from pprint import pprint
currencies = response.json()
pprint(currencies)

{'Date': '2023-07-04T11:30:00+03:00',
 'PreviousDate': '2023-07-01T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2023/07/01/daily_json.js',
 'Timestamp': '2023-07-03T20:00:00+03:00',
 'Valute': {'AED': {'CharCode': 'AED',
                    'ID': 'R01230',
                    'Name': 'Дирхам ОАЭ',
                    'Nominal': 1,
                    'NumCode': '784',
                    'Previous': 24.0633,
                    'Value': 24.3195},
            'AMD': {'CharCode': 'AMD',
                    'ID': 'R01060',
                    'Name': 'Армянских драмов',
                    'Nominal': 100,
                    'NumCode': '051',
                    'Previous': 22.894,
                    'Value': 23.0827},
            'AUD': {'CharCode': 'AUD',
                    'ID': 'R01010',
                    'Name': 'Австралийский доллар',
                    'Nominal': 1,
                    'NumCode': '036',
                    'Previous': 58.5989,
             

Теперь данные находятся в словаре и можно легко получать необходимые значения.

Например, по ключу Valute мы можем обратиться к вложенному словарю, который содержит информацию о мировых валютах. Выведем на экран, например, информацию о евро (EUR):

In [14]:
pprint(currencies['Valute']['EUR'])

{'CharCode': 'EUR',
 'ID': 'R01239',
 'Name': 'Евро',
 'Nominal': 1,
 'NumCode': '978',
 'Previous': 96.0195,
 'Value': 97.2582}


In [15]:
# Повторите запросы, описанные в этом юните, на своём компьютере. 
# Что будет выведено на экран в результате выполнения следующего кода?
# print(currencies['Valute']['CZK']['Name'])

print(currencies['Valute']['CZK']['Name'])

Чешских крон


## 4. Парсинг сайтов

Для примера рассмотрим [страницу](https://nplus1.ru/news/2021/10/11/econobel2021), содержащую статью с информацией о присуждении Нобелевской премии по экономике в 2021 году, и попробуем извлечь из неё заголовок статьи, опубликованной на странице, дату публикации, а также текст статьи.

Получить содержимое страницы в большинстве случаев несложно, гораздо труднее извлечь из HTML-кода нужную информацию.

### ОСНОВЫ HTML

HTML (англ. HyperText Markup Language, рус. язык гипертекстовой разметки) — стандартизированный язык разметки документов в интернете. Большинство веб-страниц содержат описание разметки на языке HTML. Язык HTML интерпретируется браузерами. Полученный в результате интерпретации текст отображается на экране монитора компьютера или мобильного устройства.

HTML является близким родственником уже знакомого вам формата XML. Разметка на языке HTML делается с помощью так называемых тегов, которые помещаются в угловые скобки, и применяется к элементам, заключённым внутри них. Посмотрите на примеры:

< h2 > Это заголовок второго уровня < / h2 >
< div > А это обычный текст < / div >


У корректной HTML-страницы есть заголовок и тело страницы. В заголовке (в тегах < head > … < / head >)  размещается техническая информация, подключаются скрипты и стили. В теле < body > … < / body > находятся текст и данные, которые непосредственно отображаются на странице в браузере


Пример страницы можно увидеть в файле page.html


Обратите внимание, что теги образуют иерархическую структуру, то есть одни теги расположены внутри других. В примере выше тег < p > … < / p > находится внутри тега < body > … < / body >.

Кроме того, у тегов могут быть атрибуты, которые пишутся внутри открывающегося тега. Самые популярные атрибуты — это class и id:

< h1 id="big-title" > Заголовок страницы < / h1 >
< p class="red-back" > Какой-то текст < / p >


Изучение языка HTML находится вне рамок этого курса, но для того, чтобы собирать информацию с веб-страниц, нет необходимости хорошо знать HTML. Достаточно понимать, что:

* существуют теги с разными именами;
* у тегов бывают атрибуты, такие как class и id;
* теги образуют иерархическую структуру, то есть одни теги вложены в другие.

ДОПОЛНИТЕЛЬНО

Вы можете ознакомиться с информацией о HTML в справочнике, перейдя по [ссылке](http://htmlbook.ru/html).

### ПОЛУЧАЕМ СОДЕРЖИМОЕ ВЕБ-СТРАНИЦЫ

Получим HTML-код интересующей нас страницы.

Для этого отправим GET-запрос с помощью библиотеки requests и метода get() и посмотрим на текст ответа на наш запрос (как мы помним, он содержится в атрибуте text):

In [16]:
import requests
url = 'https://nplus1.ru/news/2021/10/11/econobel2021'
response = requests.get(url)
print(response.text)

<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#" lang="ru">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>
    <link rel="preload" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" as="font" type="font/woff2" crossorigin />
    <link rel="preload" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" as="font" type="font/woff2" crossorigin />
  <link href="/front-build/css/main.css?id=c066269c9e31b8c5719ba747306ba601" rel="stylesheet">
  <link href="/front-build/css/app.css?id=09c66ab9053b6ee68d87437dc6b9d719" rel="stylesheet">
  

  <link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png">
  <link rel="icon" type="image/png" sizes="32x32" href="/favicon-32x32.png">
  <link rel="icon" type="image/png" sizes="16x16" href="/favicon-16x16.png">
  <link rel="

Ответ содержит HTML-код страницы, к которой мы обратились.
В отличие от предыдущего примера, где ответ возвращался в JSON-формате, мы не можем так просто преобразовать HTML-код в словарь и извлечь необходимую нам информацию.

Для решения таких задач в Python существует специальная библиотека __BeautifulSoup__, о работе с которой мы поговорим в следующем юните.

## 5. Библиотека BeautifulSoup

Для поиска необходимых нам данных мы будем использовать библиотеку BeautifulSoup, которая позволяет по названию тегов и их атрибутов получать содержащийся в них текст.

BeautifulSoup не является частью стандартной библиотеки, поэтому для начала её нужно установить. 

In [17]:
!pip install beautifulsoup4

In [18]:
# После установки импортируем библиотеку в наш код:
from bs4 import BeautifulSoup

Теперь мы можем извлекать данные из любой веб-страницы.

Ранее мы уже получили содержимое страницы с помощью GET-запроса и сохранили информацию в переменной response , теперь создадим объект BeautifulSoup с именем page, указывая в качестве параметра html.parser.

Для примера получим информацию o title (с англ. заголовок) — это строка, которая отображается на вкладке браузера:

In [19]:
page = BeautifulSoup(response.text, 'html.parser')
print(page.title)
print(page.title.text)

<title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>
Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей


Если при запросе к сайту, а затем при его разборе с помощью BeautifulSoup в тексте страницы не находится нужный тег, попробуйте вывести на печать пару тысяч символов текста страницы. Если там обнаружится нечто похожее на капчу, возможно, сайт посчитал вас роботом и отказывается выдавать содержимое. Чтобы получить его, попробуйте «притвориться» браузером при запросе из скрипта:

__requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})__
User-Agent своего браузера можно узнать по этой [ссылке](https://whatmyuseragent.com/)


### ИЗВЛЕКАЕМ ЗАГОЛОВОК И ВРЕМЯ НАПИСАНИЯ СТАТЬИ

Выполним поставленную ранее задачу: получить информацию о странице и извлечь заголовок статьи, опубликованной на этой странице, дату публикации, а также текст статьи.

Предположим, что мы знаем, что в HTML-коде рассматриваемой нами страницы заголовок статьи заключён в тег < h1 > … < / h1 > (заголовок первого уровня).

Тогда мы можем получить его текст с помощью метода find() (с англ. найти) объекта BeautifulSoup, передав ему название интересующего нас тега:

In [20]:
print(page.find('h1').text)


            Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей
          


Но как же узнать, в каких именно тегах заключена необходимая информация?

Проще всего это сделать с помощью так называемого инструмента разработчика, который есть во всех современных браузерах. Покажем, как открыть данный инструмент на примере использования браузера Google Chrome.

Устанавливаем курсор на элементе [страницы](https://nplus1.ru/news/2021/10/11/econobel2021) (заголовок статьи), информацию о котором хотим получить, нажимаем на правую клавишу мыши и в выпадающем списке выбираем пункт «Просмотреть код элемента» или «Посмотреть код» в зависимости от браузера.

В Microsoft Edge - кнопочка "проверить" при наведении на текст и нажатии правой кнопки мыши

Напишите функцию wiki_header, которая по адресу страницы возвращает заголовок первого уровня для статей на Wikipedia.

Функция wiki_header принимает один аргумент - url.

In [25]:
from bs4 import BeautifulSoup
import requests
def wiki_header(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    result = page.find('h1').text
    return result

print(wiki_header('https://en.wikipedia.org/wiki/Operating_system'))

Operating system


### НЕУНИКАЛЬНЫЕ ТЕГИ: ИЗВЛЕКАЕМ ТЕКСТ И ДАТУ ПУБЛИКАЦИИ СТАТЬИ

Теперь получим сам текст статьи. Как вы уже знаете, первым делом необходимо определить, в какой тег он заключён. Применим, как и ранее, инструмент разработчика.

Видим, что искомый текст заключён в тег  < div > … < / div > . Попробуем извлечь его уже известным нам способом — с помощью метода find() — и выведем его на экран.

In [26]:
response = requests.get('https://nplus1.ru/news/2021/10/11/econobel2021')
page = BeautifulSoup(response.text, 'html.parser')

print(page.find('div').text)


























N + offline
N + production




                О нас
              

                Сложность
              

                Рекламодателям
              

                Авторам
              


Физика
Зоология
Астрономия
Генетика
Математика
Космонавтика


Археология
Нейронауки
На мышах
Звук
Красота
Научные закрытия


ИИ спешит на помощь
Когда рассеется дым
Книжная полка







































О нас
Сложность
Рекламодателям
Вакансии


Физика
Зоология
Астрономия
Генетика
Математика
Космонавтика
Археология
Нейронауки
На мышах
Звук
Красота
Научные закрытия
ИИ спешит на помощь
Когда рассеется дым
Книжная полка



N + offline
N + production












































13:04


11.10.21





1.1


Экономика


Нобелевская премия























            Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей
          


Илья Ферапонтов




Премия Шведского национального банка по 

Дело в том, что теги < div > …  < / div > очень распространённые и на странице их очень много. Метод find() нашёл первый из них, но это не то, что нам надо.

Посмотрим на нашу страницу, используя инструмент разработчика, ещё раз. Можем заметить, что у искомого текста есть свой класс — n1_material text-18

Передадим название класса в метод find() с помощью аргумента class_ и получим текст статьи:

In [27]:
print(page.find('div', class_='n1_material text-18').text)

Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад в эмпирические исследования экономики рынка труда, а также Джошуа Энгристу (Joshua Angrist) и Гвидо Имбенсу (Guido Imbens) за их вклад в методологию анализа причинно-следственных связей. Прямая трансляция церемонии объявления лауреатов шла на официальном сайте Нобелевской премии.


В данном случае происходит поиск точного строкового значения class атрибута, т. е. выполнение строк кода:

__print(page.find('div', class_='n1_material').text)__
__print(page.find('div', class_='n1_material text-18').text)__
даст одинаковый результат.

При выполнении строки кода

__print(page.find('div', class_='text-18 n1_material').text)__
мы получим ошибку, так как такого строкового значения в области поиска нет.

Аналогично получим информации о теге, который содержит дату написания статьи, отображаемую в левом верхнем углу страницы.

Итак, нам нужен тег < a > … < / a > с классом "relative before:block before:w-px before:bg-current before:h-4 before:absolute before:left-0 group pl-2 flex inline-flex items-center". Для поиска достаточно указать в качестве класса "relative", отбросив дополнительные настройки.

In [28]:
print(page.find('a', class_='relative').text)


11.10.21



О поиске по классу можно узнать подробнее в [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-by-css-class)

Задача решена — мы извлекли из контента страницы заголовок статьи, опубликованной на странице, дату публикации, а также текст статьи.

### СБОР НЕСКОЛЬКИХ ЭЛЕМЕНТОВ: СОБИРАЕМ ВСЕ ССЫЛКИ НА СТРАНИЦЕ

Рассмотрим ещё один сценарий: вы хотите собрать сразу несколько элементов со страницы. Например, представьте, что вы хотите получить названия всех языков программирования, упомянутых на странице в [Wikipedia](https://en.wikipedia.org/wiki/List_of_programming_languages) в статье про языки программирования.

Можно заметить, что все названия языков программирования на этой странице связаны ссылками c соответствующими статьями о них. Таким образом, нам необходимо собрать все ссылки на странице. Для ссылок в HTML предусмотрен тег < a > … < / a >. Попробуем использовать find():

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages'
response = requests.get(url) 
page = BeautifulSoup(response.text, 'html.parser')
print(page.find('a'))

Мы получили только одну ссылку, хотя на странице их явно больше.

Это происходит, потому что метод find() возвращает только первый подходящий элемент. Если требуется получить больше элементов, необходимо воспользоваться методом find_all() (с англ. найти все):

In [31]:
links = page.find_all('a')
print(len(links)) # на момент написания ссыло 954, со временем может стать больше

954


In [32]:
print([link.text for link in links[500:510]]) # некоторые ссылки

['MAD', 'MAD/I', 'Magik', 'Magma', 'Máni', 'Maple', 'MAPPER', 'MARK-IV', 'Mary', 'MATLAB']


Не все ссылки соответствуют названиям языков программирования — страница содержит также «служебные» ссылки, такие, например, как Jump to navigation (с англ. Перейти к навигации) или Alphabetical (с англ. По алфавиту):

In [34]:
print([link.text for link in links[0:10]])

['Jump to content', 'Main page', 'Contents', 'Current events', 'Random article', 'About Wikipedia', 'Contact us', 'Donate', 'Help', 'Learn to edit']


Для обработки полученных данных и исключения «лишней» информации можно, например, использовать подходы, которые вы изучили в модуле PYTHON-14.

✍ В заключение заметим, что BeautifulSoup — достаточно мощная библиотека. Мы рассмотрели её базовые возможности, но их полный список гораздо шире. С ним можно ознакомиться в [официальной документации.](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## 6. Работа с API

В предыдущих разделах мы собирали полезную информацию с различных сайтов.

Вы могли убедиться, что поиск необходимой информации с выделением правильных тегов — довольно трудоёмкая задача. Кроме того, подобные программы могут ломаться в случаях, когда меняется дизайн сайта, его разметка или владельцы сайтов защищаются от ботов капчей.

К счастью, многие крупные сайты предоставляют доступ к так называемым API (англ. Application Programming Interface, рус. Интерфейс Прикладного Программирования).

API — это специальные разделы сайта, где информацию можно получать без разметки, а формат запросов и ответов зафиксирован. API созданы для того, чтобы облегчить взаимодействие с сайтом для сторонних разработчиков.

Рассмотрим на примере социальной сети ВКонтакте особенности API, характерные для более крупных сайтов.

### КЛЮЧ АВТОРИЗАЦИИ

Для того чтобы начать работать с API, обычно необходимо получить сервисный ключ авторизации — токен.

Токен — это средство идентификации пользователя или отдельного сеанса работы в компьютерных сетях и приложениях. Различают программные и аппаратные токены.
Мы будем использовать программный токен, который обычно представляет собой зашифрованную последовательность символов, позволяющую точно идентифицировать объект и определить уровень его привилегий. Он генерируется системой авторизации и привязывается к конкретному сеансу работы, клиенту сети или пакету данных.


Авторизация применяется практически во всех API, чтобы отдавать данные только их владельцу или контролировать количество запросов в единицу времени.

Сервисный токен для API ВКонтакте для нашей задачи создаётся вместе с новым приложением. Приложение мы делать, конечно, не будем. Оно нужно только для получения токена, чтобы сделать необходимые выгрузки.

__Как получить токен в ВК__
1. Зайдите на [страницу](https://vk.com/editapp?act=create), чтобы создать приложение (вы должны быть авторизованы ВКонтакте). Дайте приложению любое название и в разделе Платформа поставьте отметку выбора напротив значения Standalone-приложение
2. Подтвердив создание приложения на сайте в приложении ВКонтакте или по СМС, зайдите в Настройки. Нужный нам токен лежит в поле Сервисный ключ доступа


У меня нет аккаунта в ВК. Что делать?
В отличие от многих других социальных сетей, ВК предоставляет API для парсинга данных — это открывает огромные возможности для аналитиков и дата-сайентистов. На хакатонах порой бывают задачи найти какие-то интересные закономерности именно в информации из ВК.

Для работы с данными этой социальной сети достаточно создать базовый профиль с минимумом личной информации, а при регистрации потребуется ввести только номер телефона и адрес электронной почты.

Безусловно, регистрация в ВК — ваше личное дело. Однако, отказавшись от неё, вы лишите себя возможности исследовать огромные объёмы актуальных данных.

### ПЕРВЫЕ ЗАПРОСЫ К API

Чтобы познакомиться с работой API, мы будем получать данные для статистических отчётов произвольной группы, например данные о соотношении мужчин и женщин, статистику географии пользователей и т. п.

Перейдите по следующей ниже ссылке в браузере, подставив вместо слова TOKEN ваш персональный сервисный ключ доступа (токен), полученный на предыдущем шаге:

https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=TOKEN

Примечание: адресная строка в браузере будет выглядеть примерно так:  https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=8b3341297d3341297d334129917d5e4be377d337d33412920ba951f8120284cd53ff3bd

Итак, мы сделали GET-запрос к API ВКонтакте, который состоит из следующих элементов:

* https://api.vk.com/method — домен и URL запроса API; обычно не меняется;
* users.get — название метода, который отдаёт определённый отчёт, в нашем случае это метод для получения информации о пользователе;
* user_id и v — параметры запроса: идентификатор пользователя, о котором хотим получить информацию (в нашем примере мы запрашиваем информацию о первом пользователе), и номер версии API;
* token — токен, который выдаётся только пользователям, имеющим право просматривать определённые данные, например показания счётчиков Яндекс.Метрики вашего проекта; на все остальные запросы без корректного токена система отвечает отказом.

Если мы обратимся к документации метода [users.get](https://vk.com/dev/users.get), то увидим, что в ней описано множество других параметров, которые можно получить о пользователе (дата рождения, пол, родной город и другие) — словом, всё то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал).

Добавим к запросу дату рождения и пол (согласно документации, эти параметры надо перечислять в поле fields):

https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=TOKEN

Примечание: значение 2 у параметра sex означает мужской пол.

До этого момента мы делали запросы в браузере — теперь давайте выполним запросы из кода.

Новая ссылка будет иметь вид:
https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=8b3341297d3341297d334129917d5e4be377d337d33412920ba951f8120284cd53ff3bd

### ЗАПРОС К API ИЗ КОДА

In [35]:
import requests
token = '8b3341297d3341297d334129917d5e4be377d337d33412920ba951f8120284cd53ff3bd'
url = 'https://api.vk.com/method/users.get' 
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'} # Перечисляем параметры нашего запроса в словаре params
response = requests.get(url, params=params)
print(response.text)

{"response":[{"id":1,"bdate":"10.10.1984","sex":2,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false}]}


Мы получили строку в JSON-формате, которую можно преобразовать в словарь с помощью метода json(), после чего можно с лёгкостью обращаться к различным полям.

Словари нагляднее выводить с помощью функции pprint(), которую мы уже использовали ранее

In [36]:
from pprint import pprint
pprint(response.json())

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


Как вы видите, по ключу response мы можем получить список, в котором хранятся словари, содержащие информацию о запрошенных нами пользователях. Мы запросили информацию лишь об одном из них, поэтому список содержит только один элемент. Извлечём его:

In [37]:
user = response.json()['response'][0]

print(user['bdate'])

10.10.1984


Метод users.get() позволяет запрашивать информацию о множестве (до 1 000) пользователей одновременно. Для этого нужно использовать параметр user_ids и передавать id через запятую в строковом формате. Например, чтобы получить информацию о пользователях с id=1, id=2, id=3, необходимо передать значение параметра user_ids='1,2,3'.

In [39]:
ids = ','.join(map(str, range(1, 4)))
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bdate', 'access_token': token, 'lang': 'ru'}
pprint(requests.get(url, params=params).json())

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'bdate': '14.2',
               'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'can_access_closed': True,
               'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'is_closed': False,
               'last_name': ''}]}


Используя API, определите долю женщин (sex=1) среди пользователей с id от 1 до 500. Иногда будут попадаться пользователи, у которых пол не указан (sex=0), — таких пользователей не нужно учитывать в общем числе.

В ответе укажите число, округлив до двух знаков после точки-разделителя, например, 0.55.

In [49]:
ids = ','.join(map(str, range(1, 501)))
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'}

users = requests.get(url, params=params).json()['response']

total_count = 0
sex_female = 0
for page in range(len(users)):
    if users[page]['sex'] != 0:
        total_count += 1
    if users[page]['sex'] == 1:
        sex_female += 1
print(f'Общее число женщин, среди первых пользователей - {round(sex_female/total_count, 2)}')


Общее число женщин, среди первых пользователей - 0.48


### СБОР ИНФОРМАЦИИ ИЗ ГРУПП

В одном из предыдущих юнитов в качестве примера мы собрали информацию о небольшом количестве пользователей. Теперь перейдём к более реальной задаче — сбору данных о пользователях группы ВКонтакте.

Стоит отметить, что есть много сервисов, которые выгружают похожую статистику из соцсетей. Однако им свойственны недостатки универсальных решений:
* не учитываются все особенности вашего проекта;
* используется фиксированный набор метрик, дополнительную обработку данных приходится делать вам;
* не всегда бесплатны и вряд ли позволят работать с большими объёмами данных.

Теперь мы научимся считать произвольные метрики групп, собирая данные из API и работая с двумя ограничениями, которые свойственны практически всем системам:
* ограничение на количество вызовов в единицу времени;
* ограничение на количество выгружаемых строк за один запрос.

Ограничение на количество запросов в единицу времени сделано для того, чтобы избежать чрезмерной нагрузки на серверы системы. В ряде случаев небольшое количество отчётов можно выгрузить и уложиться в действующий лимит.

Однако второе ограничение не удастся обойти в случае выгрузки больших отчётов. Например, чтобы получить список всех пользователей популярной группы, серверу, возможно, придётся отправить ответ, содержащий миллионы записей.



Давайте рассмотрим, как работать с этими ограничениями на примере выгрузки списка пользователей группы https://vk.com/vk социальной сети ВКонтакте.

Обратимся к [документации](https://vk.com/dev/groups), чтобы узнать, какие методы нам доступны для групп, — для получения списка пользователей группы доступен метод [groups.getMembers](https://vk.com/dev/groups.getMembers).

Согласно документации, обязательным параметром данного метода является group_id — идентификатор, или короткое имя, группы. В нашем случае это vk: https://vk.com/vk. Протестируем, как работает метод в самом простом случае, — получим id участников группы:

In [53]:
import requests
token = '8b3341297d3341297d334129917d5e4be377d337d33412920ba951f8120284cd53ff3bd'
url = 'https://api.vk.com/method/groups.getMembers'
params = {'group_id': 'vk', 'v': 5.95, 'access_token': token}
response = requests.get(url, params = params)
data = response.json()
print(data)

{'error': {'error_code': 28, 'error_msg': 'Application authorization failed: access_token has expired.', 'request_params': [{'key': 'group_id', 'value': 'vk'}, {'key': 'v', 'value': '5.95'}, {'key': 'method', 'value': 'groups.getMembers'}, {'key': 'oauth', 'value': '1'}]}}


По ключу count мы можем получить общее число участников группы, а список по ключу items хранит их id. Посмотрим на него поближе:

In [54]:
print(len(data['response']['items']))

KeyError: 'response'

Мы видим, что всего пользователей в группе больше 11 миллионов, а получили мы только первую тысячу пользователей группы. По информации, указанной в документации о параметре count, это максимум, который может отдать API за один раз.

Для получения следующей тысячи пользователей можно воспользоваться параметром offset (с англ. смещение), который передвинет начало отсчёта. Для выгрузки всех пользователей группы будем в цикле выгружать по 1000 пользователей (count будет всегда равен 1000), увеличивая смещение offset на величину count.

Для тренировки напишем цикл выгрузки первых 20 пользователей со значением count=5. Иными словами, мы будем выгружать по пять пользователей за запрос до тех пор, пока не получим информацию о 20 пользователях.

In [55]:
users_for_cheking = data['response']['items'][:20]
print(users_for_cheking)

KeyError: 'response'

Теперь используем count и offset, чтобы получить те же id по пять за раз:

In [56]:
# не забываем импортировать requests и указать токен

url = 'https://api.vk.com/method/groups.getMembers'
count = 5
offset = 0
users_ids = []
max_count = 20
while offset < max_count:
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params)
    data = response.json()
    users_ids += data['response']['items'] # Увеличиваем смещение на количество строк, которое мы уже выгрузили 
    offset += count
print(users_ids) 

Выгружаю 5 пользователей с offset = 0


KeyError: 'response'

### ОГРАНИЧЕНИЕ ПО ЧАСТОТЕ ЗАПРОСОВ

В API часто добавляют ограничение по частоте запросов, чтобы отдельно взятые пользователи слишком сильно не перегружали сервер. Подобное ограничение есть и у ВКонтакте — в документации указано, что можно делать не более трёх запросов в секунду.

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, так как периодичность отправки запросов будет искусственно замедлена.

Воспользуемся библиотекой time и методом sleep, с помощью которого мы можем добавить паузу, например в 0.5 секунд, после каждого запроса:

In [57]:
# import requests # Импортируем модуль requests
# import time # Импортируем модуль time
# token = ' ... ' # Указываем свой сервисный токен
# url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес страницы, к которой делаем запрос
# count = 1000 
# offset = 0  
# user_ids = []  
# while offset < 5000: 
#     params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
#     response = requests.get(url, params = params) 
#     data = response.json() 
#     user_ids += data['response']['items'] 
#     offset += count 
#     print('Ожидаю 0.5 секунды...') 
#     time.sleep(0.5) 
# print('Цикл завершен, offset =',offset) 

KeyError: 'response'

### ЛАЙКИ, РЕПОСТЫ И КОММЕНТАРИИ

Через API новостной ленты ВКонтакте мы можем получить информацию о взаимодействии с сообщениями в ленте.

Для примера продолжим работать с группой https://vk.com/vk и рассмотрим последние 100 сообщений в новостной ленте.

Примечание: обратите внимание, что т.к. сообщения в новостной ленте непрерывно обновляются, то ваш результат выполнения кода ниже будет отличаться от нашего варианта.

In [58]:
from pprint import pprint # Импортируем функцию pprint()
url = 'https://dev.vk.com/method/wall.get' # Указываем адрес страницы, к которой делаем запрос
params = {'domain': 'vk', 'filter': 'owner', 'count': 1000, 'offset': 0, 'access_token': token, 'v': 5.95} 
response = requests.get(url, params = params) 
pprint(response.json()) 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [59]:
# Посмотрим на количество результатов:

len(response.json()['response']['items'])
## 100

# Посмотрим на информацию об отдельном сообщении:

response.json()['response']['items'][0] 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

В полях comments, likes и reposts содержится статистика по взаимодействию с сообщением пользователей (на момент получения информации) — число комментариев, лайков и репостов.

Давайте соберём итоговую статистику для последних десяти непустых сообщений в словарь stats. В качестве ключа будем использовать начало сообщения (если начало сообщения пустое, то информацию о таком сообщении проигнорируем), в качестве значения — список с тремя интересующими нас метриками и временем публикации (комментарии, лайки, репосты, дата публикации):

In [60]:
stats = {} 
count_post = 0 # Счётчик «непустых» сообщений
for record in response.json()['response']['items'][:]:
    title = record['text'][:30] 
    if title: 
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date']] 
        count_post += 1 
    if count_post < 10: 
        continue 
    else: 
        break 
pprint(stats)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Мы рассмотрели базовое взаимодействие с пользователями и группами. ВКонтакте предоставляет достаточно широкие возможности в своём API: всё, что можно делать вручную через браузер, доступно и в API.

ДОПОЛНИТЕЛЬНО

Если вы размещаете рекламу во ВКонтакте, то можно выгружать всю статистику через [ads API](https://vk.com/dev/ads).

Полный список методов ВКонтакте можно посмотреть в [документации](https://vk.com/dev/methods).

### ДРУГИЕ API


API для разработчиков предоставляют и многие другие платформы. Вот список, пожалуй, самых популярных из них:

[Google Maps API](https://developers.google.com/maps/)
[YouTube API](https://developers.google.com/youtube/)
[Twitter API](https://dev.twitter.com/overview/documentation)
[Facebook API](https://developers.facebook.com/docs/)


Информацию о сторонних API можно найти в [каталоге Web API](https://www.programmableweb.com/category/all/apis). Также можно воспользоваться интернет-поиском, указав в строке поиска, например, «курсы валют API» или «прогноз погоды api», — среди первых результатов выдачи чаще всего с лёгкостью можно найти ссылки на необходимый функционал.

## 7. Как настроить регулярную выгрузку данных

Как уже говорилось, скриптом принято называть небольшую компьютерную программу, которая автоматизирует выполнение некоторой задачи. Программы, которые мы создаём на языке Python, также являются скриптами.

Скрипт можно использовать разными способами:

* Если вам нужно выполнить код один раз или выполнять его очень редко (от случая к случаю), вы можете запустить скрипт вручную из командной строки.
* Если вы, например, написали на Python серверную часть веб-сайта и хотите, чтобы ваш код мог в любое время обрабатывать запросы, вы захотите запустить его как фоновый процесс.
* А что делать, если вам нужно, чтобы ваш скрипт запускался иногда? Каждую пятницу, 13-го? В день рождения супруга (или супруги)? Или просто каждый час?

В этом случае вам нужен автоматический запуск скриптов, или, как часто его называют программисты, запуск по крону — от английского акронима Cron (англ. Command Run ON) — названия системы для автоматического запуска программ и скриптов на сервере в определённое время.

Автоматический запуск может понадобиться, например:

* если вы хотите с определённой периодичностью скачивать новую информацию с сайтов, например выполнять парсинг новостей для последующего анализа (как мы уже знаем, этот процесс называется web-scraping);
* если ваш скрипт должен следить за курсом акций и каждую минуту делать запрос по API, чтобы получить новые котировки;
* если вы написали обучающую платформу и вам нужно каждый час проверять, кто из студентов приступил к занятиям и насколько успешно продвигается их обучение;
* если у вас есть 500 вендинговых автоматов по продаже солнечных очков и каждые пять минут вы должны опрашивать все автоматы, чтобы узнать, не закончились ли очки.

В каждом, даже небольшом, проекте обычно есть 5-10 скриптов, которые с определённой периодичностью обновляют сертификаты доменов, очищают место на диске, считают статистику, генерируют отчёты и так далее. Давайте научимся настраивать запуск этих скриптов в соответствии с задачами, которые стоят перед вами.

### КАК НАСТРОИТЬ АВТОМАТИЧЕСКИЙ ЗАПУСК

Во всех операционных системах есть встроенные средства запуска программ по расписанию. Однако можно задать расписание запуска необходимых вам функций на языке Python внутри скрипта.

модуль schedule (c англ. расписание). Он позволяет запускать код как с определённым интервалом, так и в заданное время.

Модуль schedule не входит в стандартную библиотеку Python, поэтому его необходимо установить:

In [64]:
!pip install schedule
import schedule

### ПОСТАНОВКА ЗАДАЧИ

Рассмотрим вариант автоматического запуска простой функции, которая выводит на экран короткое сообщение:

In [65]:
def task():
    print('Hello! I am a task!')
    return

Предположим, что мы хотим, чтобы функция запускалась через определённые интервалы времени, а именно каждые 15 минут.

Для запуска задачи через определённые интервалы времени в модуле schedule используется метод every(), который получает в качестве единственного аргумента число, указывающее, как часто следует запускать код.

Далее вызывается метод, определяющий единицы измерения промежутков времени, через которые будет выполняться функция. В нашем примере это минуты. Вот как будет выглядеть итоговый код:

In [68]:
schedule.every(15).seconds.do(task)

Если бы мы хотели запускать задачу, например, каждый час, то могли бы написать:

schedule.every(1).hour.do(task) 

In [69]:
schedule.cancel_job(task)

### ВЫПОЛНЕНИЕ ФУНКЦИИ

После того как мы создали нашу функцию и определились со временем её запуска, мы можем запустить наш менеджер расписания (schedule). Для этого надо создать бесконечный цикл. Да-да, именно так, ведь скрипт должен выполняться постоянно, чтобы постоянно проверять, не пришло ли время снова выполнить задачу.

Внутри цикла мы будем вызывать особый метод run_pending() для объекта schedule, который будет проверять, нет ли задачи, которую пора выполнить.

После вызова метода run_pending() нужно будет сделать небольшую паузу, после которой можно будет снова проверять, не пришло ли время для выполнения какой-либо функции.

In [71]:
import time
while True:
    schedule.run_pending()
    time.sleep(1)

Hello! I am a task!
Hello! I am a task!
Hello! I am a task!
Hello! I am a task!


KeyboardInterrupt: 

Всё готово — теперь можно сохранить файл и выполнить код.

Вот какой код в итоге получился:

import schedule

def task(): 
    print('Hello! I am a task!') 
    return

schedule.every(15).minutes.do(task)

import time 
while True: 
    schedule.run_pending() 
    time.sleep(1)
    
Этот код будет каждую секунду проверять, не надо ли выполнить какую-то задачу, и раз в 15 минут будет выводить на экран фразу: "Hello! I am a task!" Вывод сообщения будет повторяться до тех пор, пока вы не остановите выполнение скрипта.

ПАРСИНГ САЙТОВ

[Официальная документация BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

РАБОТА С API

[Описание методов API ВКонтакте](https://vk.com/dev/methods)

АВТОМАТИЧЕСКИЙ ЗАПУСК СКРИПТОВ

[Документация модуля schedule](http://schedule.readthedocs.io/)

[Как работать с планировщиком заданий в ОС Windows](https://www.comss.ru/page.php?id=4840)

[О том, что такое CRON и как его использовать](https://tproger.ru/translations/guide-to-cron-jobs/)

# Что если? Я чувствую, что мне не хватает материалов курса для полного понимания языка Python.

Поскольку это курс по Data Science, Python здесь даётся как необходимый, но далеко не единственный инструмент и изучается обзорно, ведь должно остаться время и на высшую математику, статистику, алгоритмы машинного обучения.

Материала по Python в курсе достаточно для успешного старта в DS. В дальнейшем в процессе работы вам могут потребоваться более глубокие знания — пробелы можно и нужно восполнять самостоятельно, ведь без этого будет невозможным ваше развитие как специалиста.

В материалах курса приводятся ссылки на документацию, где вы можете подробнее ознакомиться с первоисточником, а также на дополнительные материалы, чтобы при желании вы могли глубже погрузиться в конкретную тему.

При возникновении вопросов по содержанию курса вы всегда можете обратиться за помощью к менторам в Пачке. Однако если вы убеждены, что материалов по какой-то теме недостаточно, напишите об этом в обратной связи по модулю — мы внимательно следим за ОС от наших студентов и учитываем её при дальнейшей доработке курса.


# Для аттестации

Советуем вам ознакомиться со [статьёй](https://nuancesprog.ru/p/11460/), где собрано 80 самых популярных вопросов по Python, которые задают на собеседованиях.